## Import Necessary Dependencies


In [1]:
# OS
import os

# Time
import time

#CSV 
import csv

# Excel Sheets
import csv

# ###########
#   Modules - Classes that contain functions used within code
# #############
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

# Beautiful Soup
from bs4 import BeautifulSoup
# # Login Module ( For Using Login Class )
# from modules.login import Login as login
# # Go To Homepage Module ( Making Company Search Query and Navigating to Homepage )
# from modules.go_to_homepage import GoToHomepage
# # Get Search Module 
# from modules.get_search import GetSearch
# # Get Search Module 
# from modules.collect import Collect


# CONFIG
import configparser
import pathlib
config = configparser.ConfigParser()
config.read("config.ini")
EMAIL = config["DEFAULT"]["EMAIL"]
KEY = config["DEFAULT"]["KEY"]



driver = Service('driver/chromedriver')
browser = webdriver.Chrome(service=driver)

## Add Companies and Queries (Search Terms)
 

In [5]:
# GET COMPANIES
companies = []
file_path = "company_urls.txt"
with open(file_path, 'r') as companies_list:
    for i in companies_list.readlines():
        name= i.replace('\n', "")
        companies.append(name)


# SEARCH TERMS
search_terms = ['marketing', 'strategy', 'operations']

## Set headers for Exported CSV

In [2]:
# Set Excel Header Row
headers = ["Company URL", "marketing", "strategy", "operations"]

with open('final_list.csv', 'a') as f:
    object = csv.writer(f)
    object.writerow(headers)
    f.close()

## Login

In [3]:
# Login
def login(username, key):
    browser.get('https://www.linkedin.com/uas/login')
    elementID = browser.find_element(By.ID, 'username')
    elementID.send_keys(username)
    elementID = browser.find_element(By.ID, 'password')
    elementID.send_keys(key)
    elementID.submit()   

login(EMAIL, KEY)

## Search Function

In [6]:
class Search():

    def __init__(self, browser, search_terms, time):
        self.browser = browser
        self.search_terms = search_terms
        self.time = time


    # Go To Homepage Method
    def go_to_page(self, company_url):
        self.browser.get(company_url)
        self.browser.implicitly_wait(2)

        data = []
        for search_term in self.search_terms:
            results = self.get_search(company_url, search_term)
            data.append(results)
            
        marketing=data[0]
        strategy=data[1]
        operations=data[2]

        row = [company_url, marketing, strategy, operations]

        with open("final_list.csv", 'a') as csvfile: 
            object = csv.writer(csvfile)
            object.writerow(row)
            csvfile.close()

    def get_search(self, company, search_query):
        self.time.sleep(2) 
        #Navigate to company homepage by clicking on search query link. 
        #Use direct selenium to grab company employee search query page 

        try: 
            company_search_query_page_parent_element = browser.find_element(
                By.CLASS_NAME, 'display-flex.mt2.mb1')
            company_search_query_page = company_search_query_page_parent_element.find_element(By.CLASS_NAME,
            'ember-view')
            company_search_query_page.click()
            self.time.sleep(3)
            search_query_url = str(self.browser.current_url)
            position = search_query_url.find('COMPANY_PAGE')
            base_emp_search_url = search_query_url[:position]
            route = f'{base_emp_search_url}FACETED_SEARCH&title={search_query}'
            self.browser.get(route)
            self.time.sleep(3)
            employees = self.collect(company)
        except NoSuchElementException: 
            employees = []
            browser.get(company)
            self.time.sleep(2)             
        
        return employees
        

    def collect(self, company):
        self.time.sleep(3)
        try:
            # Load page content using beautiful soup
            src = browser.page_source
            soup = BeautifulSoup(src)
            ul_tag = soup.find(
                'ul', {'class': 'reusable-search__entity-result-list list-style-none'})
            li_tags = ul_tag.find_all('li')
            # Loop over each list item and collect data
            employees = []
            for li_tag in li_tags:
                # [Company, Search Query, Name, Title, Location, Employee ]
                # Find name and profile link ( Same element )
                name_parent = li_tag.find(
                    'span', {'class', 'entity-result__title-text t-16'})
                target_element = name_parent.find("a", href=True)
                name = target_element.get_text().strip()
                page_link = target_element['href']
                
                # Find title
                title = li_tag.find(
                    'div', {'class': 'entity-result__primary-subtitle t-14 t-black t-normal'}).get_text().strip()

                # Find location
                location = li_tag.find(
                    'div', {'class': 'entity-result__secondary-subtitle t-14 t-normal'}).get_text().strip()
                
                employee = {
                    name: name, 
                    title: title, 
                    page_link: page_link, 
                    location: location
                }
                employees.append(employee)

            # print(employees)
            browser.get(company)
            self.time.sleep(2)            
            return employees

        except:
            employees = []
            self.time.sleep(2)
            return employees

        # Navigate back to company homepage 


#Login
search = Search(browser, search_terms, time)
for company_url in companies: 
    search.go_to_page(company_url)
    
browser.close()

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=98.0.4758.102)
Stacktrace:
0   chromedriver                        0x00000001015c4e69 chromedriver + 5160553
1   chromedriver                        0x000000010154f593 chromedriver + 4679059
2   chromedriver                        0x00000001011032c8 chromedriver + 172744
3   chromedriver                        0x00000001010f2a05 chromedriver + 104965
4   chromedriver                        0x00000001010f3eb2 chromedriver + 110258
5   chromedriver                        0x00000001010ecba2 chromedriver + 80802
6   chromedriver                        0x0000000101104753 chromedriver + 178003
7   chromedriver                        0x000000010116875c chromedriver + 587612
8   chromedriver                        0x0000000101156623 chromedriver + 513571
9   chromedriver                        0x000000010112bdce chromedriver + 339406
10  chromedriver                        0x000000010112d105 chrom